<center>
    <h1>CMSC320 Final Project</h1>
    Valentyn Bolotin, Jacob Barats, Evan May
</center>
<hr>

<center>
    <h3>Introduction</h3>
</center>

Since its creation, alcohol has been a staple in human society. Every country at some point in history has drank alochol and today most countries still do. The difference between then and now is the change in restrictions to alochol. While few countries like Kuwait have <a href="https://www.latimes.com/world/middleeast/la-fg-kuwait-islamists-alcohol-20150105-story.html#:~:text=Kuwait%20and%20Saudi%20Arabia%20are,people%20convicted%20of%20drinking%20alcohol">absolute bans on alcohol</a>, many have regulations in place to limit the amount of alcohol consumption for their citizens. The exact reasoning for these restirctions change depending on the country but the general thought process is that when individuals consume less alcohol, they are better, more productive citizens. 

We want to challenge this reasoning by determining if a countries alochol consumption impacts a countries gross domestic product (GDP). While a country's GDP is not a perfect relection on the productivity of a country, there are enough coorelations between the two that analysis can be conducted.

<hr>

<center>
    <h3>Data collection/curation</h3>
</center>


To determine if alcohol consumption impacts a country's GDP, we need to wrangle data for a country's alcohol consumption and GDP. The problem was that many of these datasets were incomplete or over a small time frame. Our solution was wrangling data from several sources and filling out as much missing information as possible. We have listed all the dataset that we use in this tutorial below.  

* Our World in Data
    * <a href = "https://ourworldindata.org/alcohol-consumption">Alochol Consumption per Capita</a>
    * <a href = "https://ourworldindata.org/grapher/gdp-world-regions-stacked-area?tab=table&time=earliest..1991&country=GRC~DEU~GHA~GTM~GNB~GIN~GMB~GAB~GNQ~GEO">Gross Domestic Product (GDP) based on 2011 International Dollar ($)</a>
    * <a href = "https://ourworldindata.org/grapher/alcohol-consumption-vs-gdp-per-capita">Alochol Consumption per Capita vs GDP per Capita based on Purchasing Power Partiry (PPP) </a>
<br> 
* University of Groningen, Maddison Project Database 2018
    * <a href = "https://www.rug.nl/ggdc/historicaldevelopment/maddison/releases/maddison-project-database-2018">GDP per capita based on 2011 US Dollar with and without multiple benchmarks, Total Population </a>
<br> 
* The World Bank, International Comparison Program (ICP)
    * <a href = "https://ourworldindata.org/alcohol-consumption">Purchasing Power Partiry (PPP)</a>
<br>
* World Health Organization
    * <a href = "https://www.who.int/data/gho/data/indicators/indicator-details/GHO/alcohol-recorded-per-capita-(15-)-consumption-(in-litres-of-pure-alcohol)">Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)</a>

<hr>

<center>
    <h3>Parsing and Data management</h3>
</center>
To complete the tutorial the following imports are required: Pandas, Numpy
<br>
The warnings import is just to avoid unncessary output and it will still run without it.

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas.core.arrays import integer
from pandas.core.arrays.numeric import T
import numpy as np

<h4>Finding the Best Range</h4>

As stated previously, the data we aquired had large amounts of missing data. Many countries did not recorded data for various reasons meaning the datasets contain sections of time where there is little to no meaningful data. If we were to take all of the data and preform analysis on it, it would be be too skewed by the missing data to produce any meaningful results. Our decison was to determine the range of years that had the most data.

First we added the dataset from the Maddison Project Database. 

In [27]:
raw_data = pd.read_stata('Data/mpd2018.dta')
raw_df = pd.DataFrame(raw_data)
raw_df = raw_df.rename(columns= {"countrycode" : "code", "pop":"population",
                          "rgdpnapc":"gdp_pc_2011", "cgdppc":"gdp_pc_2011_mb"})
raw_df = raw_df.drop(columns=['i_cig', 'i_bm'])
combined_df = raw_df
combined_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population
0,AFG,Afghanistan,1820.0,NaN,NaN,3280.0
1,AFG,Afghanistan,1870.0,NaN,NaN,4207.0
2,AFG,Afghanistan,1913.0,NaN,NaN,5730.0
3,AFG,Afghanistan,1950.0,2392.0,2392.0,8150.0
4,AFG,Afghanistan,1951.0,2422.0,2422.0,8284.0
...,...,...,...,...,...,...
19868,ZWE,Zimbabwe,2012.0,1623.0,1604.0,12620.0
19869,ZWE,Zimbabwe,2013.0,1801.0,1604.0,13183.0
19870,ZWE,Zimbabwe,2014.0,1797.0,1594.0,13772.0
19871,ZWE,Zimbabwe,2015.0,1759.0,1560.0,14230.0


(**DESCRIPTION OF HOW THIS WORKS**)
(**EXPLAIN MEANING OF OUTPUT**)
* Index :
* Year :
* absDvt : 
* dv_ser : 
* dt :
* adj_percent : 

In [30]:
codes_arr = []

for code in pd.unique(combined_df.code):
  if isinstance(code, str):
    codes_arr.append(code)
combined_df = combined_df[combined_df.code.isin(codes_arr)]

years_arr = np.sort(pd.unique(combined_df.year).reshape(-1))

years_num = {}

for year in years_arr:
  num_entries = len(combined_df[combined_df.year == year].index)
  years_num[year] = num_entries

for i in range(0, len(years_arr)):
  year = years_arr[i]
  prev_year = year
  if i > 0:
    prev_year = years_arr[i-1]
  delta_t = year - prev_year
  delta_v = 0
  delta_vt = 0
  if delta_t > 0:
    delta_v = (years_num[year] - years_num[prev_year][0])
    delta_vt = float(delta_v)/float(delta_t)
  years_num[year] = [years_num[year], years_num[year] - len(codes_arr), delta_t,
                     delta_v, delta_vt]

high_delta_indicies = [] ##[index, delta_abs, series, time_delta, %elts ser, num_entry]

##we are going to assume the data trends up, so lowest offset from max countries
##is the metric we are going to use for large delta jumps, a change in ~20% of 
##data points should do the trick

start_year = years_arr[0]
high_delta_indicies.append([0, np.abs(years_num[start_year][4]), 0, 0, 
          1.0 + float(years_num[start_year][1])/float(len(codes_arr)),
          1])
hd_index = 0

for i in range(1, len(years_arr)):
  year = years_arr[i]
  curr_dvt = np.abs(years_num[year][4])
  curr_dt = years_num[year][2]
  curr_dv = years_num[year][3]
  if(curr_dvt > 0.2 * float(years_num[year][0])):
    hd_index = hd_index + 1
    high_delta_indicies.append([i, curr_dvt, 0, curr_dt,
      1.0 + float(years_num[year][1])/float(len(codes_arr)), 1])
  else:
    tmp = high_delta_indicies[hd_index]
    tmp[2] = tmp[2] + curr_dv
    tmp[3] = tmp[3] + curr_dt
    tmp[4] = tmp[4] + 1.0 + float(years_num[year][1])/float(len(codes_arr))
    tmp[5] = tmp[5] + 1
    high_delta_indicies[hd_index] = tmp


#lets remove any singular data points, theyre not useful

tmp = []
for elt in high_delta_indicies:
  if elt[5] > 1:
    tmp.append(elt)

high_delta_indicies = tmp

#our output
for elt in high_delta_indicies:
  to_print = "Index: {}, year: {}, absDvt: {}, dv_ser: {}, dt: {}, adj_percent: {}"
  to_print = to_print.format(elt[0], years_arr[elt[0]], elt[1], elt[2], elt[3],
                             elt[4]/elt[3])
  print(to_print)

Index: 0, year: 1.0, absDvt: 0, dv_ser: -43, dt: 1279.0, adj_percent: 0.00048577152083497164
Index: 5, year: 1281.0, absDvt: 3.0, dv_ser: 0, dt: 19.0, adj_percent: 0.00591715976331364
Index: 25, year: 1301.0, absDvt: 4.0, dv_ser: 0, dt: 47.0, adj_percent: 0.011834319526627278
Index: 73, year: 1349.0, absDvt: 5.0, dv_ser: 0, dt: 51.0, adj_percent: 0.011834319526627278
Index: 125, year: 1401.0, absDvt: 6.0, dv_ser: 0, dt: 8.0, adj_percent: 0.011834319526627265
Index: 133, year: 1409.0, absDvt: 1.0, dv_ser: 0, dt: 41.0, adj_percent: 0.0177514792899407
Index: 175, year: 1451.0, absDvt: 4.0, dv_ser: 0, dt: 49.0, adj_percent: 0.0177514792899407
Index: 225, year: 1501.0, absDvt: 45.0, dv_ser: 0, dt: 29.0, adj_percent: 0.0177514792899407
Index: 254, year: 1530.0, absDvt: 1.0, dv_ser: 0, dt: 20.0, adj_percent: 0.023668639053254552
Index: 275, year: 1551.0, absDvt: 4.0, dv_ser: 0, dt: 19.0, adj_percent: 0.023668639053254552
Index: 295, year: 1571.0, absDvt: 5.0, dv_ser: 0, dt: 24.0, adj_percent:

Based on the analysis done we can now score each of the ranges and find the most suitable range to conduct our analysis on.

Using the information collected, we now can calculate the scores for each of the year ranges. 

In [31]:
##lets score all of our values
hd_scores = []

for elt in high_delta_indicies:
  score = 0
  ## % covered
  score += elt[4]/elt[3]
  ## ~time step
  time_step = elt[3]/elt[5]
  time_step_score = np.exp(-(time_step - 1))
  score += time_step_score
  ## total entries
  score += elt[5]/len(years_arr)
  ## delta trend
  dt_score = float(elt[2])/float(len(codes_arr))
  dt_score = dt_score * elt[4]/elt[3]
  dt_score = dt_score/2.0 + 0.5
  score += dt_score
  hd_scores.append(score)

max_index = np.argmax(hd_scores)

print("Best index: {}, score: {}, start_year: {}, end_year: {}".format(
  max_index, hd_scores[max_index], years_arr[high_delta_indicies[max_index][0]],
  years_arr[high_delta_indicies[max_index][0]] + high_delta_indicies[max_index][3]
  - years_num[years_arr[high_delta_indicies[max_index][0]]][2]
))

Best index: 27, score: 2.5904183535762484, start_year: 1950.0, end_year: 2016.0


For our tutorial we will use data from 1950 to 2016. This will make later analysis easier as there is a smaller likelihood of missing data. We can now remove the unnessesary rows from our dataframe, which is around 8000 rows. 

In [32]:
start_year = years_arr[high_delta_indicies[max_index][0]]
end_year = start_year + high_delta_indicies[max_index][3]

combined_df = combined_df[combined_df.year >= start_year]
combined_df = combined_df[combined_df.year <= end_year]

combined_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population
3,AFG,Afghanistan,1950.0,2392.0,2392.0,8150.0
4,AFG,Afghanistan,1951.0,2422.0,2422.0,8284.0
5,AFG,Afghanistan,1952.0,2462.0,2462.0,8425.0
6,AFG,Afghanistan,1953.0,2568.0,2568.0,8573.0
7,AFG,Afghanistan,1954.0,2576.0,2576.0,8728.0
...,...,...,...,...,...,...
19868,ZWE,Zimbabwe,2012.0,1623.0,1604.0,12620.0
19869,ZWE,Zimbabwe,2013.0,1801.0,1604.0,13183.0
19870,ZWE,Zimbabwe,2014.0,1797.0,1594.0,13772.0
19871,ZWE,Zimbabwe,2015.0,1759.0,1560.0,14230.0


<h4>Finding the missing values</h4>
While the range of years we have found has the least amount of missing entries, there is still a relative small amount in our data. In this tutorial we want to make sure that we have as much GDP data as possible. To fill in this data we will extract more data from the Our World in Data dataset.

In [33]:
cpc_df = pd.read_csv('Data/alcohol-consumption-vs-gdp-per-capita.csv')
cpc_df = pd.DataFrame(cpc_df)

cpc_df = cpc_df.drop(labels = "Continent", axis = 1)
cpc_df = cpc_df.rename(columns= {"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)" : "consumption_for_person",
                                 "GDP per capita, PPP (constant 2017 international $)" : "gdp_pc_2017", "Population (historical estimates)": "population_cpc",
                                 "Entity" : "country", "Code":"code", "Year":"year"})
cpc_df = cpc_df[cpc_df.code.isin(codes_arr)]
cpc_df["total_liters_consumed"] = cpc_df.consumption_for_person * cpc_df.population_cpc
cpc_df = cpc_df[cpc_df.year > start_year]
future_cpc_df = cpc_df[cpc_df.year > end_year]

cpc_df = cpc_df[cpc_df.year <= end_year]
cpc_df.gdp_pc_2017 = cpc_df.gdp_pc_2017
cpc_df

,country,code,year,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed
1,Afghanistan,AFG,2010,0.21,1957.029070,29185511.0,6128957.31
2,Afghanistan,AFG,2015,0.21,2068.265904,34413603.0,7226856.63
4,Afghanistan,AFG,2002,NaN,1189.784668,22600774.0,NaN
5,Afghanistan,AFG,2003,NaN,1235.810063,23680871.0,NaN
6,Afghanistan,AFG,2004,NaN,1200.278013,24726689.0,NaN
...,...,...,...,...,...,...,...
57077,Zimbabwe,ZWE,1985,NaN,NaN,8877489.0,NaN
57078,Zimbabwe,ZWE,1986,NaN,NaN,9200150.0,NaN
57079,Zimbabwe,ZWE,1987,NaN,NaN,9527202.0,NaN
57080,Zimbabwe,ZWE,1988,NaN,NaN,9849129.0,NaN


We can search for missing entries in the main dataframe and attempt to fill it with the data in the new dataframe. The dataset from Our World in Data has data on total alcohol consumption per capita so we can add that to our main dataframe. Note that all missing entries from this point forward will be represented as -1.

In [36]:
hit_years_arr = np.sort(pd.unique(combined_df.year).reshape(-1))

temp_comb = combined_df

cpc_merge = cpc_df.drop(labels=["gdp_pc_2017","consumption_for_person", "population_cpc", "total_liters_consumed"],
                             axis = 1)

for code in codes_arr:
  cpc_code = cpc_merge[cpc_merge.code == code]
  combined_code = combined_df[combined_df.code == code]
  missing_years = np.array(hit_years_arr)
  sub_years = np.array(combined_code.year.to_numpy())
  missing_years = np.setdiff1d(missing_years, sub_years)
  
  if len(missing_years > 0):
    cpc_fill = cpc_code[cpc_code.year.isin(missing_years)]
    cpc_fill.index += len(temp_comb.index)
    temp_comb = pd.concat([temp_comb,cpc_fill])

cpc_merge = cpc_df.drop(labels = ["country"], axis = 1)
temp_comb = pd.merge(how = "left", left = temp_comb, right = cpc_merge, on=["code","year"])
temp_comb = temp_comb.fillna(-1)

combined_df = temp_comb
combined_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed
0,AFG,Afghanistan,1950.0,2392.0,2392.0,8150.0,-1.00,-1.000000,-1.0,-1.00
1,AFG,Afghanistan,1951.0,2422.0,2422.0,8284.0,-1.00,-1.000000,7840151.0,-1.00
2,AFG,Afghanistan,1952.0,2462.0,2462.0,8425.0,-1.00,-1.000000,7935996.0,-1.00
3,AFG,Afghanistan,1953.0,2568.0,2568.0,8573.0,-1.00,-1.000000,8039684.0,-1.00
4,AFG,Afghanistan,1954.0,2576.0,2576.0,8728.0,-1.00,-1.000000,8151316.0,-1.00
...,...,...,...,...,...,...,...,...,...,...
11318,ZWE,Zimbabwe,2012.0,1623.0,1604.0,12620.0,-1.00,3170.726522,13115149.0,-1.00
11319,ZWE,Zimbabwe,2013.0,1801.0,1604.0,13183.0,-1.00,3176.829330,13350378.0,-1.00
11320,ZWE,Zimbabwe,2014.0,1797.0,1594.0,13772.0,-1.00,3195.767970,13586710.0,-1.00
11321,ZWE,Zimbabwe,2015.0,1759.0,1560.0,14230.0,4.92,3198.982129,13814642.0,67968038.64


We only want data that has a low amount of missing entries. Data that is missing all of its population data isn't helpful so we remove them.

In [40]:
missing_codes = {}
for code in codes_arr:
  pop = combined_df[combined_df.code == code]
  pop = pop[((pop.population < 0) | (pop.population_cpc < 0))]
  pop_arr = []
  for i, row in pop.iterrows():
    pop_arr.append(row.year)
  missing_codes[code] = pop_arr

def code_comp (a):
  return -len(missing_codes[a])

sorted_codes = sorted(codes_arr, key=code_comp)

rem_codes = []
for code in sorted_codes:
  if len(missing_codes[code]) == end_year - start_year:
    rem_codes.append(code)

combined_df = combined_df[~combined_df.code.isin(rem_codes)]
hit_codes = pd.unique(combined_df.code).reshape(-1)
combined_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed
0,AFG,Afghanistan,1950.0,2392.0,2392.0,8150.0,-1.00,-1.000000,-1.0,-1.00
1,AFG,Afghanistan,1951.0,2422.0,2422.0,8284.0,-1.00,-1.000000,7840151.0,-1.00
2,AFG,Afghanistan,1952.0,2462.0,2462.0,8425.0,-1.00,-1.000000,7935996.0,-1.00
3,AFG,Afghanistan,1953.0,2568.0,2568.0,8573.0,-1.00,-1.000000,8039684.0,-1.00
4,AFG,Afghanistan,1954.0,2576.0,2576.0,8728.0,-1.00,-1.000000,8151316.0,-1.00
...,...,...,...,...,...,...,...,...,...,...
11318,ZWE,Zimbabwe,2012.0,1623.0,1604.0,12620.0,-1.00,3170.726522,13115149.0,-1.00
11319,ZWE,Zimbabwe,2013.0,1801.0,1604.0,13183.0,-1.00,3176.829330,13350378.0,-1.00
11320,ZWE,Zimbabwe,2014.0,1797.0,1594.0,13772.0,-1.00,3195.767970,13586710.0,-1.00
11321,ZWE,Zimbabwe,2015.0,1759.0,1560.0,14230.0,4.92,3198.982129,13814642.0,67968038.64


<h4>Additional tidying</h4>
We want to make sure that all of the data is in the same units. We change our population's scale to be one to one and make both our GDP columns be in terms of GDP per capita. This avoids problems of unit conversion in analysis later.

In [41]:
st_df = combined_df

st_df["population"] = st_df["population"] * 1000
st_df.population[st_df.population < 0] = -1

st_df["gdp_2011"] = st_df["gdp_pc_2011"] * st_df["population"]
st_df.gdp_2011[st_df.gdp_2011 < 0] = -1

st_df["gdp_2011_mb"] = st_df["gdp_pc_2011_mb"] * st_df["population"]
st_df.gdp_2011_mb[st_df.gdp_2011_mb < 0] = -1

st_df["gdp_2017"] = st_df["gdp_pc_2017"] * st_df["population_cpc"]
st_df.gdp_2017[st_df.gdp_2017 < 0] = -1

st_df["population"] = st_df["population"].astype(int)
st_df["population_cpc"] = st_df["population_cpc"].astype(int)
st_df["year"] = st_df["year"].astype(int)
st_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,gdp_2011,gdp_2011_mb,gdp_2017
0,AFG,Afghanistan,1950,2392.0,2392.0,8150000,-1.00,-1.000000,-1,-1.00,1.949480e+10,1.949480e+10,1.000000e+00
1,AFG,Afghanistan,1951,2422.0,2422.0,8284000,-1.00,-1.000000,7840151,-1.00,2.006385e+10,2.006385e+10,-1.000000e+00
2,AFG,Afghanistan,1952,2462.0,2462.0,8425000,-1.00,-1.000000,7935996,-1.00,2.074235e+10,2.074235e+10,-1.000000e+00
3,AFG,Afghanistan,1953,2568.0,2568.0,8573000,-1.00,-1.000000,8039684,-1.00,2.201546e+10,2.201546e+10,-1.000000e+00
4,AFG,Afghanistan,1954,2576.0,2576.0,8728000,-1.00,-1.000000,8151316,-1.00,2.248333e+10,2.248333e+10,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11318,ZWE,Zimbabwe,2012,1623.0,1604.0,12620000,-1.00,3170.726522,13115149,-1.00,2.024248e+10,2.048226e+10,4.158455e+10
11319,ZWE,Zimbabwe,2013,1801.0,1604.0,13183000,-1.00,3176.829330,13350378,-1.00,2.114553e+10,2.374258e+10,4.241187e+10
11320,ZWE,Zimbabwe,2014,1797.0,1594.0,13772000,-1.00,3195.767970,13586710,-1.00,2.195257e+10,2.474828e+10,4.341997e+10
11321,ZWE,Zimbabwe,2015,1759.0,1560.0,14230000,4.92,3198.982129,13814642,67968038.64,2.219880e+10,2.503057e+10,4.419279e+10


The 2017 GDP that was added to our dataset is in terms of the constant 2017 internation dollar which is different than the 2011 GDP and the 2011 MB GDP. This causes issues when they are compared to one another in an analytical setting. Conversion factors are calculated for 2011 and 2016 are calculated for 2011, 2011 with multiple benchmarks, and 2017.

In [44]:
##Fill missing economic datapoints
##First find conversion factors

y1_df = combined_df[combined_df.year == 2011]
y2_df = combined_df[combined_df.year == 2016]
y1_vals = {"gdp_2011":0, "gdp_2011_mb":0, "gdp_2017":0}
y2_vals = {"gdp_2011":0, "gdp_2011_mb":0, "gdp_2017":0}
for code in hit_codes:
  c_df = y1_df[combined_df.code == code]
  for v in c_df["gdp_2011"]:
    y1_vals["gdp_2011"] = y1_vals["gdp_2011"] + v
  for v in c_df["gdp_2011_mb"]:
    y1_vals["gdp_2011_mb"] = y1_vals["gdp_2011_mb"] + v
  for v in c_df["gdp_2017"]:
    y1_vals["gdp_2017"] = y1_vals["gdp_2017"] + v
  c_df = y2_df[combined_df.code == code]
  for v in c_df["gdp_2011"]:
    y2_vals["gdp_2011"] = y2_vals["gdp_2011"] + v
  for v in c_df["gdp_2011_mb"]:
    y2_vals["gdp_2011_mb"] = y2_vals["gdp_2011_mb"] + v
  for v in c_df["gdp_2017"]:
    y2_vals["gdp_2017"] = y2_vals["gdp_2017"] + v

y1_vals["fac1"] = y1_vals["gdp_2011"] / y1_vals["gdp_2017"]
y1_vals["fac2"] = y1_vals["gdp_2011_mb"] / y1_vals["gdp_2017"]
y1_vals["fac2011"] = y1_vals["gdp_2011_mb"] / y1_vals["gdp_2011"]

y2_vals["fac1"] = y2_vals["gdp_2011"] / y2_vals["gdp_2017"]
y2_vals["fac2"] = y2_vals["gdp_2011_mb"] / y2_vals["gdp_2017"]
y2_vals["fac2011"] = y2_vals["gdp_2011_mb"] / y2_vals["gdp_2011"]

print(y1_vals)
print(y2_vals)

combined_df[combined_df.code == "USA"]

{'gdp_2011': 93952331058496.0, 'gdp_2011_mb': 93952331058496.0, 'gdp_2017': 98171539732185.94, 'fac1': 0.957022079054683, 'fac2': 0.957022079054683, 'fac2011': 1.0}
{'gdp_2011': 107295416080637.0, 'gdp_2011_mb': 106491490024125.0, 'gdp_2017': 115602680525130.58, 'fac1': 0.9281395171222896, 'fac2': 0.92118530072471, 'fac2011': 0.9925073587868114}


,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,gdp_2011,gdp_2011_mb,gdp_2017
10720,USA,United States,1950,15241.0,15241.0,152271008,-1.00,-1.000000,-1,-1.000000e+00,2.320762e+12,2.320762e+12,1.000000e+00
10721,USA,United States,1951,16126.0,16126.0,154878000,-1.00,-1.000000,160872264,-1.000000e+00,2.497563e+12,2.497563e+12,-1.000000e+00
10722,USA,United States,1952,16443.0,16443.0,157552992,-1.00,-1.000000,163266026,-1.000000e+00,2.590644e+12,2.590644e+12,-1.000000e+00
10723,USA,United States,1953,16917.0,16917.0,160184000,-1.00,-1.000000,165909996,-1.000000e+00,2.709833e+12,2.709833e+12,-1.000000e+00
10724,USA,United States,1954,16513.0,16513.0,163026000,-1.00,-1.000000,168736390,-1.000000e+00,2.692048e+12,2.692048e+12,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10782,USA,United States,2012,50394.0,50394.0,314743008,-1.00,55581.413844,314043885,-1.000000e+00,1.586116e+13,1.586116e+13,1.745500e+13
10783,USA,United States,2013,50863.0,50863.0,317072992,-1.00,56214.428677,316400539,-1.000000e+00,1.612728e+13,1.612728e+13,1.778628e+13
10784,USA,United States,2014,51664.0,51664.0,319558016,-1.00,57213.267841,318673422,-1.000000e+00,1.650965e+13,1.650965e+13,1.823235e+13
10785,USA,United States,2015,52591.0,52591.0,322073984,9.82,58540.296093,320878312,3.151025e+09,1.693819e+13,1.693819e+13,1.878431e+13


<h4>Created the world dataset</h4>
We now have data for country and year but we want to make an easy access data that has the yearly sum for many of these columns such as GDP and total alcohol consumption. We create a world dataset to accomplish this.

In [49]:
world_df = pd.DataFrame(columns= combined_df.columns)
world_miss_df = pd.DataFrame(columns= world_df.columns).drop(labels = ["country", "code"], axis = 1)

common_attrs = ["population",
                "population_cpc",
                "consumption_for_person",
                "gdp_pc_2011_mb",
                "gdp_pc_2011",
                "gdp_pc_2017",
                "total_liters_consumed",
                "gdp_2011_mb",
                "gdp_2011",
                "gdp_2017"]

frac_attrs =   ["gdp_pc_2011_mb",
                "gdp_pc_2011",
                "gdp_pc_2017",
                "consumption_for_person"]

map_attrs =    {"gdp_pc_2011_mb" : "gdp_2011_mb",
                "gdp_pc_2011" : "gdp_2011",
                "gdp_pc_2017": "gdp_2017",
                "consumption_for_person" : "total_liters_consumed"}

for year in hit_years_arr:
  year_df = combined_df[combined_df.year == year]
  ser = pd.Series()
  ser["country"] = "WORLD"
  ser["code"] = "WORLD"
  ser["year"] = year

  ser_miss = pd.Series()
  ser_miss["year"] = year
  
  for attr in common_attrs:
    ser[attr] = 0
    ser_miss[attr] = []

  for i, row in year_df.iterrows():
    for attr in common_attrs:
      if row[attr] > 0:
        ser[attr] = row[attr] + ser[attr]
      else:
        ser_miss[attr].append(row["code"])

  for attr in common_attrs:
    if ser[attr] == 0:
      ser[attr] = -1

      
  # FIX FRAC ATTRS &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  for attr in frac_attrs:
    if ser[attr] > 0:
      ser[attr] = ser[map_attrs[attr]] / ser["population"]
  world_df.loc[len(world_df.index)] = ser
  world_miss_df.loc[len(world_miss_df.index)] = ser_miss
  ##world_df = pd.concat([world_df,pd.Series(vs)], axis = 0, ignore_index = True)
  ##world_miss_df = pd.concat([world_miss_df,ser_miss], axis = 0)
world_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,gdp_2011,gdp_2011_mb,gdp_2017
0,WORLD,WORLD,1950.0,2840.316642,3193.007878,2514160984,-1,-1,-1,-1,8027735828641.0,7141013283921.0,166.0
1,WORLD,WORLD,1951.0,2985.611376,3355.141447,2557931000,-1,-1,2572198645,-1,8582220316169.0,7636987893105.0,-1
2,WORLD,WORLD,1952.0,3067.791124,3456.668876,2610876008,-1,-1,2618839562,-1,9024933836048.0,8009622243888.0,-1
3,WORLD,WORLD,1953.0,3191.232846,3591.33825,2658728016,-1,-1,2665387479,-1,9548391621048.0,8484620172448.0,-1
4,WORLD,WORLD,1954.0,3204.739537,3626.651794,2709558016,-1,-1,2712412149,-1,9826623440912.0,8683427700912.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,WORLD,WORLD,2012.0,13815.576851,13817.003207,6992922992,-1,14492.744916,7079608754,-1,96621239410112.0,96611265008512.0,101346649138136.046875
63,WORLD,WORLD,2013.0,14030.044051,14084.76023,7072213016,-1,14806.919497,7163434702,-1,99610424623616.0,99223460153152.0,104717688794875.0625
64,WORLD,WORLD,2014.0,14261.271531,14366.788468,7152269064,-1,15140.486309,7247241265,-1,102755136711424.0,102000451182720.0,108288831838648.9375
65,WORLD,WORLD,2015.0,14493.906019,14601.919766,7231374064,6.175409,15485.600274,7330861223,44656690316.553009,105591943876928.0,104810856072960.0,111982168185818.25


Using this data we can but the fraction of the GDP, alcohol consumed, and population that a country represents for a given year. This is placed in the main dataframe.

In [50]:
comb_df = pd.concat([combined_df,world_df], ignore_index = True)

comb_df["frac_gdp_2011"] = -1
comb_df["frac_gdp_2011_mb"] = -1
comb_df["frac_gdp_2017"] = -1
comb_df["frac_liters_consumed"] = -1
comb_df["frac_population"] = -1
comb_df["frac_population_cpc"] = -1

for year in hit_years_arr:
  total = world_df[world_df.year == year].gdp_2011.to_numpy()[0]
  comb_df.frac_gdp_2011[comb_df.year == year] = comb_df.gdp_2011 / total

  total = world_df[world_df.year == year].gdp_2011_mb.to_numpy()[0]
  comb_df.frac_gdp_2011_mb[comb_df.year == year] = comb_df.gdp_2011_mb / total

  total = world_df[world_df.year == year].gdp_2017.to_numpy()[0]
  comb_df.frac_gdp_2017[comb_df.year == year] = comb_df.gdp_2017 / total

  total = world_df[world_df.year == year].total_liters_consumed.to_numpy()[0]
  comb_df.frac_liters_consumed[comb_df.year == year] = comb_df.total_liters_consumed / total

  total = world_df[world_df.year == year].population.to_numpy()[0]
  comb_df.frac_population[comb_df.year == year] = comb_df.population / total

  total = world_df[world_df.year == year].population_cpc.to_numpy()[0]
  comb_df.frac_population_cpc[comb_df.year == year] = comb_df.population_cpc / total

comb_df.frac_gdp_2011[comb_df["gdp_2011"] == -1] = -1
comb_df.frac_gdp_2011_mb[comb_df["gdp_2011_mb"] == -1] = -1
comb_df.frac_gdp_2017[comb_df["gdp_2017"] == -1] = -1
comb_df.frac_liters_consumed[comb_df["total_liters_consumed"] == -1] = -1
comb_df.frac_population[comb_df["population"] == -1] = -1
comb_df.frac_population_cpc[comb_df["population_cpc"] == -1] = -1

comb_df[comb_df.code == "USA"]

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,gdp_2011,gdp_2011_mb,gdp_2017,frac_gdp_2011,frac_gdp_2011_mb,frac_gdp_2017,frac_liters_consumed,frac_population,frac_population_cpc
10586,USA,United States,1950,15241.0,15241.0,152271008,-1.0,-1.0,-1,-1.0,2320762404864.0,2320762404864.0,1.0,0.289093,0.324991,0.006024,-1,0.060565,-1
10587,USA,United States,1951,16126.0,16126.0,154878000,-1.0,-1.0,160872264,-1.0,2497562542080.0,2497562542080.0,-1.0,0.291016,0.327035,-1,-1,0.060548,0.062543
10588,USA,United States,1952,16443.0,16443.0,157552992,-1.0,-1.0,163266026,-1.0,2590643847168.0,2590643847168.0,-1.0,0.287054,0.323441,-1,-1,0.060345,0.062343
10589,USA,United States,1953,16917.0,16917.0,160184000,-1.0,-1.0,165909996,-1.0,2709832597504.0,2709832597504.0,-1.0,0.2838,0.319382,-1,-1,0.060248,0.062246
10590,USA,United States,1954,16513.0,16513.0,163026000,-1.0,-1.0,168736390,-1.0,2692048224256.0,2692048224256.0,-1.0,0.273955,0.310021,-1,-1,0.060167,0.062209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10648,USA,United States,2012,50394.0,50394.0,314743008,-1.0,55581.413844,314043885,-1.0,15861159034880.0,15861159034880.0,17455003137211.960938,0.164158,0.164175,0.172231,-1,0.045009,0.044359
10649,USA,United States,2013,50863.0,50863.0,317072992,-1.0,56214.428677,316400539,-1.0,16127283429376.0,16127283429376.0,17786275533020.800781,0.161904,0.162535,0.16985,-1,0.044834,0.044169
10650,USA,United States,2014,51664.0,51664.0,319558016,-1.0,57213.267841,318673422,-1.0,16509645619200.0,16509645619200.0,18232347846554.539062,0.16067,0.161859,0.168368,-1,0.044679,0.043972
10651,USA,United States,2015,52591.0,52591.0,322073984,9.82,58540.296093,320878312,3151025023.84,16938193387520.0,16938193387520.0,18784311394301.296875,0.160412,0.161607,0.167744,0.070561,0.044538,0.043771


Now that the GDP information is collected, we import the dataset from the World Health Organization which has data about specific types of alcohol consumption by country. We remove the rows outside of our time range.

In [18]:
acp_df = pd.read_csv("Data/who-alcohol-per-cap.csv")
acp_df = pd.DataFrame(acp_df)

acp_df = acp_df.drop(labels= ["IndicatorCode","Indicator","ValueType","ParentLocationCode","ParentLocation", "Location type","Period type"], axis = 1)
acp_df = acp_df.rename(columns={"SpatialDimValueCode" : "code"})

filtered_acp_df = pd.DataFrame(columns= ["code", "year",
                                         "wine_mi", "wine_hi", "wine_lo",
                                         "beer_mi", "beer_hi", "beer_lo", 
                                         "spirits_mi", "spirits_hi", "spirits_lo", 
                                         "other_mi", "other_hi", "other_lo",
                                         "total_mi", "total_hi", "total_lo"])

for code in pd.unique(acp_df.code):
  c_df = acp_df[acp_df.code == code]
  for year in pd.unique(c_df.Period):
    y_df = c_df[c_df.Period == year]
    ser = pd.Series(index=filtered_acp_df.columns)
    ser["code"] = code
    ser["year"] = year
    got_ser = 0

    got_df = y_df[y_df["Dim1ValueCode"] == "SA_WINE"]
    if len(got_df.index > 0):
      got_ser = got_df.iloc[0]
      ser["wine_hi"] = got_ser["FactValueNumericHigh"]
      ser["wine_mi"] = got_ser["FactValueNumeric"]
      ser["wine_lo"] = got_ser["FactValueNumericLow"]

    got_df = y_df[y_df["Dim1ValueCode"] == "SA_BEER"]
    if len(got_df.index > 0):
      got_ser = got_df.iloc[0]
      ser["beer_hi"] = got_ser["FactValueNumericHigh"]
      ser["beer_mi"] = got_ser["FactValueNumeric"]
      ser["beer_lo"] = got_ser["FactValueNumericLow"]

    got_df = y_df[y_df["Dim1ValueCode"] == "SA_SPIRITS"]
    if len(got_df.index > 0):
      got_ser = got_df.iloc[0]
      ser["spirits_hi"] = got_ser["FactValueNumericHigh"]
      ser["spirits_mi"] = got_ser["FactValueNumeric"]
      ser["spirits_lo"] = got_ser["FactValueNumericLow"]

    got_df = y_df[y_df["Dim1ValueCode"] == "SA_OTHER_ALCOHOL"]
    if len(got_df.index > 0):
      got_ser = got_df.iloc[0]
      ser["other_hi"] = got_ser["FactValueNumericHigh"]
      ser["other_mi"] = got_ser["FactValueNumeric"]
      ser["other_lo"] = got_ser["FactValueNumericLow"]

    got_df = y_df[y_df["Dim1ValueCode"] == "SA_TOTAL"]
    if len(got_df.index > 0):
      got_ser = got_df.iloc[0]
      ser["total_hi"] = got_ser["FactValueNumericHigh"]
      ser["total_mi"] = got_ser["FactValueNumeric"]
      ser["total_lo"] = got_ser["FactValueNumericLow"]

    filtered_acp_df.loc[len(filtered_acp_df.index)] = ser

hit_acp_df = filtered_acp_df[filtered_acp_df.code.isin(hit_codes)]
hit_acp_df = hit_acp_df[hit_acp_df.year.isin(hit_years_arr)]

hit_acp_df = hit_acp_df.fillna(-1)
hit_acp_df

,code,year,wine_mi,wine_hi,wine_lo,beer_mi,beer_hi,beer_lo,spirits_mi,spirits_hi,spirits_lo,other_mi,other_hi,other_lo,total_mi,total_hi,total_lo
0,AFG,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.014,0.006,0.0,0.0,0.0,0.01,0.014,0.006
1,AFG,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.014,0.006,0.0,0.0,0.0,0.01,0.014,0.006
2,AFG,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.014,0.006,0.0,0.0,0.0,0.01,0.014,0.006
3,AFG,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.014,0.006,0.0,0.0,0.0,0.01,0.014,0.006
4,AFG,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10086,SDN,1965,0.01,NaN,NaN,1.19,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,1.22,NaN,NaN
10087,SDN,1964,0.01,NaN,NaN,1.51,NaN,NaN,0.03,NaN,NaN,NaN,NaN,NaN,1.54,NaN,NaN
10088,SDN,1963,0.0,NaN,NaN,1.44,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,1.47,NaN,NaN
10089,SDN,1962,0.0,NaN,NaN,1.66,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,1.68,NaN,NaN


(**EXPLAIN SIGNIFICANCE OF DOING THIS**) (**EXPLAIN OUTPUT**)

In [51]:
miss_year = pd.DataFrame(columns=["year",
                                  "miss_wine", "miss_wine_range",
                                  "miss_beer", "miss_beer_range", 
                                  "miss_spirits", "miss_spirits_range", 
                                  "miss_other", "miss_other_range",
                                  "miss_total", "miss_total_range"])

##format-> series
##keys-> miss_beer
##keys-> miss_beer_range .. for all types

for year in hit_years_arr:
  year_df = hit_acp_df[hit_acp_df.year == year]
  if len(year_df.index) < 1:
    0==0
    ##miss_year[year] = -1
  else:
    offset = len(hit_codes) - len(year_df)
    ser = pd.Series(index = miss_year.columns)
    ser["miss_wine"] = len(year_df[year_df.wine_mi == -1].index) + offset
    ser["miss_wine_range"] = len(year_df[(year_df.wine_hi == -1) | (year_df.wine_lo == -1)].index) + offset
    ser["miss_beer"] = len(year_df[year_df.beer_mi == -1].index) + offset
    ser["miss_beer_range"] = len(year_df[(year_df.beer_hi == -1) | (year_df.beer_lo == -1)].index) + offset
    ser["miss_spirits"] = len(year_df[year_df.spirits_mi == -1].index) + offset
    ser["miss_spirits_range"] = len(year_df[(year_df.spirits_hi == -1) | (year_df.spirits_lo == -1)].index) + offset
    ser["miss_other"] = len(year_df[year_df.other_mi == -1].index) + offset
    ser["miss_other_range"] = len(year_df[(year_df.other_hi == -1) | (year_df.other_lo == -1)].index) + offset
    ser["miss_total"] = len(year_df[year_df.total_mi == -1].index) + offset
    ser["miss_total_range"] = len(year_df[(year_df.total_hi == -1) | (year_df.total_lo == -1)].index) + offset
    ser["year"] = year
    miss_year.loc[len(miss_year.index)] = ser


miss_year["score"] = 200 - (25 * miss_year['miss_beer']/len(hit_codes) +
                            5 * miss_year['miss_beer_range']/len(hit_codes) +
                            25 * miss_year['miss_wine']/len(hit_codes) + 
                            5 * miss_year['miss_wine_range']/len(hit_codes) + 
                            25 * miss_year['miss_spirits']/len(hit_codes) + 
                            5 * miss_year['miss_spirits_range']/len(hit_codes) + 
                            10 * miss_year['miss_other']/len(hit_codes) + 
                            5 * miss_year['miss_other_range']/len(hit_codes) + 
                            90 * miss_year['miss_total']/len(hit_codes) + 
                            5 * miss_year['miss_total_range']/len(hit_codes))

miss_year.head()

,year,miss_wine,miss_wine_range,miss_beer,miss_beer_range,miss_spirits,miss_spirits_range,miss_other,miss_other_range,miss_total,miss_total_range,score
0,1960.0,164.0,166.0,164.0,166.0,164.0,166.0,166.0,166.0,164.0,166.0,1.987952
1,1961.0,40.0,166.0,40.0,166.0,40.0,166.0,123.0,166.0,40.0,166.0,127.831325
2,1962.0,38.0,166.0,38.0,166.0,38.0,166.0,123.0,166.0,38.0,166.0,129.819277
3,1963.0,31.0,166.0,31.0,166.0,31.0,166.0,122.0,166.0,31.0,166.0,136.837349
4,1964.0,31.0,166.0,31.0,166.0,31.0,166.0,122.0,166.0,31.0,166.0,136.837349


We then extract the viable data entries and merge them to our main dataframe. The rows are merged if they have the same code and year.

In [52]:
viable_years = miss_year[miss_year.score > 180].year.to_numpy()
viable_acp_df = hit_acp_df[hit_acp_df.year.isin(viable_years)]
new_comb_df = pd.merge(how = "left", left = comb_df[comb_df.year.isin(viable_years)], right = viable_acp_df, on=["code","year"])
new_comb_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,...,beer_lo,spirits_mi,spirits_hi,spirits_lo,other_mi,other_hi,other_lo,total_mi,total_hi,total_lo
0,AFG,Afghanistan,2000,735.0,735.0,22461000,-1.0,-1.0,20779957,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,2001,692.0,692.0,22507000,-1.0,-1.0,21606992,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,2002,1088.0,1088.0,23600000,-1.0,1189.784668,22600774,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,2003,1112.0,1112.0,25005000,-1.0,1235.810063,23680871,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,2004,1107.0,1107.0,25698000,-1.0,1200.278013,24726689,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834,WORLD,WORLD,2012.0,13815.576851,13817.003207,6992922992,-1,14492.744916,7079608754,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2835,WORLD,WORLD,2013.0,14030.044051,14084.76023,7072213016,-1,14806.919497,7163434702,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2836,WORLD,WORLD,2014.0,14261.271531,14366.788468,7152269064,-1,15140.486309,7247241265,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2837,WORLD,WORLD,2015.0,14493.906019,14601.919766,7231374064,6.175409,15485.600274,7330861223,44656690316.553009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Using the information from the from the <a href = "https://www.who.int/data/gho/data/indicators/indicator-details/GHO/alcohol-recorded-per-capita-(15-)-consumption-(in-litres-of-pure-alcohol)">World Health Organization</a>, we can calculate values of volume per type of alcohol consumed. For our purposes the average for wine, beer, spirits, and other were taken to use against the data collected in the previous section.

In [53]:
# From the World Health Organization
wine_abv = [0.12,0.09,0.16]
beer_abv = [0.05]
spirits_abv = [0.4,0.3]
other_abv = [0.05,0.05,0.17,0.18,0.09,0.09,0.09]

wine_abv = np.mean(wine_abv)
beer_abv = np.mean(beer_abv)
spirits_abv = np.mean(spirits_abv)
other_abv = np.mean(other_abv)

new_comb_df["wine_mi_vol"] = new_comb_df["wine_mi"] / wine_abv
new_comb_df["wine_hi_vol"] = new_comb_df["wine_hi"] / wine_abv
new_comb_df["wine_lo_vol"] = new_comb_df["wine_lo"] / wine_abv
new_comb_df["beer_mi_vol"] = new_comb_df["beer_mi"] / beer_abv
new_comb_df["beer_hi_vol"] = new_comb_df["beer_hi"] / beer_abv
new_comb_df["beer_lo_vol"] = new_comb_df["beer_lo"] / beer_abv
new_comb_df["spirits_mi_vol"] = new_comb_df["spirits_mi"] / spirits_abv
new_comb_df["spirits_hi_vol"] = new_comb_df["spirits_hi"] / spirits_abv
new_comb_df["spirits_lo_vol"] = new_comb_df["spirits_lo"] / spirits_abv
new_comb_df["other_mi_vol"] = new_comb_df["other_mi"] / other_abv
new_comb_df["other_hi_vol"] = new_comb_df["other_hi"] / other_abv
new_comb_df["other_lo_vol"] = new_comb_df["other_lo"] / other_abv

new_comb_df["total_mi_vol"] = new_comb_df["wine_mi_vol"] + new_comb_df["beer_mi_vol"] + new_comb_df["spirits_mi_vol"] + new_comb_df["other_mi_vol"]
new_comb_df["total_hi_vol"] = new_comb_df["wine_hi_vol"] + new_comb_df["beer_hi_vol"] + new_comb_df["spirits_hi_vol"] + new_comb_df["other_hi_vol"]
new_comb_df["total_lo_vol"] = new_comb_df["wine_lo_vol"] + new_comb_df["beer_lo_vol"] + new_comb_df["spirits_lo_vol"] + new_comb_df["other_lo_vol"]

new_comb_df["consumption_for_person"][new_comb_df.consumption_for_person < 0] = new_comb_df["total_mi"]
new_comb_df["fluid_vol_mi"] = new_comb_df["total_mi_vol"] * new_comb_df["population"]
new_comb_df["vol_vs_pure_mi"] = new_comb_df["total_mi"] / new_comb_df["total_mi_vol"]
new_comb_df[new_comb_df.code == "USA"]

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,...,spirits_hi_vol,spirits_lo_vol,other_mi_vol,other_hi_vol,other_lo_vol,total_mi_vol,total_hi_vol,total_lo_vol,fluid_vol_mi,vol_vs_pure_mi
2686,USA,United States,2000,45887.0,45887.0,282737984,9.19,50124.891177,281710914,2588923299.66,...,9.600000,4.085714,0.0,0.0,0.0,108.800772,150.572973,66.161390,30762110989.689571,0.075459
2687,USA,United States,2001,45878.0,45878.0,285550016,8.25,50126.720222,284607992,-1.0,...,9.771429,4.400000,0.0,0.0,0.0,109.600772,154.268726,66.637838,31296502255.542858,0.075273
2688,USA,United States,2002,46267.0,46267.0,288212000,8.33,50528.788577,287279312,-1.0,...,9.600000,4.114286,0.0,0.0,0.0,110.249421,153.908108,67.514286,31775206081.853279,0.075556
2689,USA,United States,2003,47158.0,47158.0,290700000,8.4,51529.726404,289815567,-1.0,...,10.114286,4.485714,0.0,0.0,0.0,109.316602,150.265637,68.610039,31778336293.436287,0.076841
2690,USA,United States,2004,48493.0,48493.0,293401984,8.48,52994.55196,292354663,-1.0,...,10.400000,4.514286,0.0,0.0,0.0,109.645560,150.081081,67.719691,32170224795.477993,0.077340
2691,USA,United States,2005,49655.0,49655.0,296119008,9.49,54353.068576,294993509,2799488400.41,...,10.771429,4.228571,0.0,0.0,0.0,109.398456,151.252510,67.482625,32394962148.546719,0.078155
2692,USA,United States,2006,50490.0,50490.0,298988000,8.63,55368.359719,297758977,-1.0,...,10.714286,4.657143,0.0,0.0,0.0,109.837066,151.125097,68.911197,32839964580.694981,0.078571
2693,USA,United States,2007,50902.0,50902.0,301846016,8.74,55873.244741,300608425,-1.0,...,10.800000,4.771429,0.0,0.0,0.0,111.075676,151.729730,68.463320,33527750177.210808,0.078685
2694,USA,United States,2008,50276.0,50276.0,304713984,8.74,55271.657467,303486022,-1.0,...,11.028571,4.800000,0.0,0.0,0.0,110.361390,152.444788,69.216216,33628658814.912743,0.079194
2695,USA,United States,2009,48453.0,48453.0,307396992,8.67,53399.365632,306307565,-1.0,...,11.028571,4.685714,0.0,0.0,0.0,108.485714,151.012355,68.145174,33348182246.399998,0.079918



<ol>
    <li>What the data means</li>
</ol>


<ol>
    <li>Explanations of what the columns mean (the important columns that we will use)</li>
    <li>Did we need to parse the data at all?</li>
    <li>Show how we checked if there were any missing values and how we handled them</li>
    <li>Did we need to tidy the data/remove unnessesary data</li>
  </ol>
<hr>

<center>
    <h3>Exploratory data analysis</h3>
</center>
<ol>
    <li>How did we want to display the data</li>
    <li>Code of displaying data</li>
    <li>Graphs/Charts of displayed data</li>
    </ol>
There will be multiple groups of these three sections
<hr>


<center>
    <h3>Hypothesis testing</h3>
</center>
<ol>
    <li>Conduct hypothesis testing</li>
    <li>Code of displaying data</li>
    <li>Show results in multiple formats (graphs, data, etc)</li>
  </ol>
<hr>

<center>
    <h3>Communication of insights attained</h3>
</center>
  <ol>
    <li>Descirbe what this data means (give multiple interpretations)</li>
    <li>Give reasoning as to why the results were like this (any error that could have influenced results?)</li>
    <li>Conclusion</li>
  </ol>
<hr>